In [1]:
%run chrome_dino_game_env.ipynb

pygame 2.1.0 (SDL 2.0.16, Python 3.7.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
Python 3.7.13


/Users/lorenzleisner/opt/anaconda3/envs/iannwtf/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)
2022-09-23 16:00:45.035403: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Assets written to: models/actor/assets
INFO:tensorflow:Assets written to: models/critic/assets
episode 0 score -8.9 avg score -8.9 time_steps 160 learning_steps 5
INFO:tensorflow:Assets written to: models/actor/assets
INFO:tensorflow:Assets written to: models/critic/assets
episode 1 score 12.4 avg score 1.8 time_steps 408 learning_steps 12
INFO:tensorflow:Assets written to: models/actor/assets
INFO:tensorflow:Assets written to: models/critic/assets
episode 2 score 68.0 avg score 23.8 time_steps 1063 learning_steps 33
episode 3 score 1.7 avg score 18.3 time_steps 1229 learning_steps 38
episode 4 score 1.5 avg score 14.9 time_steps 1394 learning_steps 43
episode 5 score -9.2 avg score 10.9 time_steps 1476 learning_steps 46
episode 6 score -9.4 avg score 8.0 time_steps 1556 learning_steps 48
episode 7 score -9.1 avg score 5.9 time_steps 1639 learning_steps 51
episode 8 score 1.6 avg score 5.4 time_steps 1805 learning_steps 56
episode 9 score -9.2 avg score 3.9 time_steps 1

episode 110 score -9.5 avg score 21.0 time_steps 33137 learning_steps 1035
episode 111 score -9.1 avg score 21.0 time_steps 33220 learning_steps 1038
episode 112 score -9.1 avg score 21.0 time_steps 33303 learning_steps 1040
episode 113 score -9.4 avg score 20.7 time_steps 33383 learning_steps 1043
episode 114 score 1.3 avg score 20.8 time_steps 33546 learning_steps 1048
episode 115 score -9.1 avg score 20.8 time_steps 33629 learning_steps 1050
episode 116 score 1.5 avg score 20.6 time_steps 33793 learning_steps 1056
episode 117 score -9.4 avg score 20.6 time_steps 33873 learning_steps 1058
episode 118 score 185.3 avg score 22.0 time_steps 35450 learning_steps 1107
episode 119 score -9.3 avg score 22.0 time_steps 35531 learning_steps 1110
episode 120 score -9.4 avg score 21.9 time_steps 35611 learning_steps 1112
episode 121 score 12.1 avg score 21.8 time_steps 35857 learning_steps 1120
episode 122 score 76.8 avg score 22.6 time_steps 36600 learning_steps 1143
episode 123 score 1.7 avg 

episode 220 score -9.3 avg score 10.4 time_steps 58754 learning_steps 1836
episode 221 score 99.5 avg score 11.3 time_steps 59673 learning_steps 1864
episode 222 score -9.3 avg score 10.4 time_steps 59755 learning_steps 1867
episode 223 score 78.1 avg score 11.2 time_steps 60487 learning_steps 1890
episode 224 score 186.3 avg score 13.1 time_steps 62051 learning_steps 1939
episode 225 score -9.6 avg score 13.1 time_steps 62127 learning_steps 1941
episode 226 score -9.0 avg score 12.8 time_steps 62211 learning_steps 1944
episode 227 score 12.3 avg score 13.0 time_steps 62458 learning_steps 1951
episode 228 score -9.3 avg score 13.0 time_steps 62540 learning_steps 1954
episode 229 score -9.0 avg score 12.9 time_steps 62624 learning_steps 1957
episode 230 score 1.6 avg score 12.7 time_steps 62789 learning_steps 1962
episode 231 score -9.3 avg score 12.7 time_steps 62871 learning_steps 1964
episode 232 score 34.2 avg score 12.7 time_steps 63287 learning_steps 1977
episode 233 score -9.2 av

...

episode 1606 score 45.4 avg score 18.9 time_steps 422869 learning_steps 13214
episode 1607 score 1.5 avg score 19.0 time_steps 423033 learning_steps 13219
episode 1608 score 11.9 avg score 19.2 time_steps 423276 learning_steps 13227
episode 1609 score 1.6 avg score 18.9 time_steps 423441 learning_steps 13232
episode 1610 score 282.3 avg score 21.9 time_steps 425763 learning_steps 13305
episode 1611 score 33.8 avg score 21.2 time_steps 426175 learning_steps 13317
episode 1612 score -9.4 avg score 21.2 time_steps 426255 learning_steps 13320
episode 1613 score 98.7 avg score 21.2 time_steps 427167 learning_steps 13348
episode 1614 score -9.2 avg score 20.9 time_steps 427250 learning_steps 13351
episode 1615 score 1.3 avg score 21.0 time_steps 427413 learning_steps 13356
episode 1616 score -9.2 avg score 21.0 time_steps 427495 learning_steps 13359
episode 1617 score 207.0 avg score 23.1 time_steps 429239 learning_steps 13413
episode 1618 score 153.5 avg score 24.8 time_steps 430550 learnin

episode 1712 score -9.0 avg score 17.9 time_steps 455079 learning_steps 14221
episode 1713 score -9.4 avg score 16.8 time_steps 455159 learning_steps 14223
episode 1714 score -9.2 avg score 16.8 time_steps 455242 learning_steps 14226
episode 1715 score 120.7 avg score 18.0 time_steps 456300 learning_steps 14259
episode 1716 score -9.6 avg score 18.0 time_steps 456379 learning_steps 14261
episode 1717 score 1.3 avg score 16.0 time_steps 456541 learning_steps 14266
episode 1718 score -9.0 avg score 14.3 time_steps 456625 learning_steps 14269
episode 1719 score 99.3 avg score 15.4 time_steps 457519 learning_steps 14297
episode 1720 score 34.2 avg score 15.9 time_steps 457935 learning_steps 14310
episode 1721 score -9.3 avg score 15.9 time_steps 458016 learning_steps 14313
episode 1722 score -9.4 avg score 15.9 time_steps 458097 learning_steps 14315
episode 1723 score 57.1 avg score 16.5 time_steps 458669 learning_steps 14333
episode 1724 score 34.9 avg score 16.6 time_steps 459069 learnin

episode 1818 score -9.4 avg score 16.5 time_steps 484322 learning_steps 15135
episode 1819 score 23.6 avg score 15.8 time_steps 484658 learning_steps 15145
episode 1820 score -9.1 avg score 15.4 time_steps 484742 learning_steps 15148
episode 1821 score -9.2 avg score 15.4 time_steps 484825 learning_steps 15150
episode 1822 score -9.4 avg score 15.4 time_steps 484905 learning_steps 15153
episode 1823 score 1.4 avg score 14.8 time_steps 485069 learning_steps 15158
episode 1824 score -9.2 avg score 14.4 time_steps 485152 learning_steps 15161
episode 1825 score -9.4 avg score 14.4 time_steps 485233 learning_steps 15163
episode 1826 score 1.8 avg score 14.5 time_steps 485400 learning_steps 15168
episode 1827 score -9.2 avg score 14.2 time_steps 485482 learning_steps 15171
episode 1828 score -9.3 avg score 14.2 time_steps 485563 learning_steps 15173
episode 1829 score -9.0 avg score 14.0 time_steps 485647 learning_steps 15176
episode 1830 score -9.1 avg score 13.5 time_steps 485730 learning_

episode 1924 score -9.4 avg score 13.0 time_steps 510264 learning_steps 15945
episode 1925 score 1.6 avg score 13.1 time_steps 510429 learning_steps 15950
episode 1926 score 282.5 avg score 15.9 time_steps 512753 learning_steps 16023
episode 1927 score -9.0 avg score 15.9 time_steps 512837 learning_steps 16026
episode 1928 score -9.4 avg score 15.9 time_steps 512918 learning_steps 16028
episode 1929 score 12.5 avg score 16.1 time_steps 513167 learning_steps 16036
episode 1930 score 2.0 avg score 16.3 time_steps 513336 learning_steps 16041
episode 1931 score -9.3 avg score 16.3 time_steps 513417 learning_steps 16044
episode 1932 score -9.2 avg score 16.2 time_steps 513499 learning_steps 16046
episode 1933 score 67.1 avg score 17.0 time_steps 514146 learning_steps 16067
episode 1934 score -9.4 avg score 16.9 time_steps 514226 learning_steps 16069
episode 1935 score -9.4 avg score 16.6 time_steps 514307 learning_steps 16072
episode 1936 score 1.5 avg score 16.7 time_steps 514472 learning_

/Users/lorenzleisner/opt/anaconda3/envs/iannwtf/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)
/Users/lorenzleisner/opt/anaconda3/envs/iannwtf/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


In [8]:
from keras.layers import Dense, Activation
from keras.models import Sequential, load_model

In [9]:
env = ChromeDinoEnv()

In [10]:
env.observation_space.sample()

array([ 474.18318284, 2302.08356864,  365.50760302, 2110.91761293])

In [11]:
class ReplayBuffer(object):
    def __init__(self, max_size, input_shape, n_actions, discrete=False):
        self.mem_size = max_size
        self.input_shape = input_shape
        self.discrete = discrete
        self.mem_cntr = 0
        self.state_memory = np.zeros((self.mem_size, input_shape))
        self.new_state_memory = np.zeros((self.mem_size, input_shape))
        dtype = np.int8 if self.discrete else np.float32
        self.action_memory = np.zeros((self.mem_size, n_actions), dtype=dtype)
        self.reward_memory = np.zeros(self.mem_size)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.float32)
        
    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.reward_memory[index] = reward
        self.terminal_memory[index] = 1-int(done)
        if self.discrete:
            actions = np.zeros(self.action_memory.shape[1])
            actions[action] = 1.0
            self.action_memory[index] = actions
        else:
            self.action_memory[index] = action
        self.mem_cntr += 1
    
    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, batch_size)
        
        states = self.state_memory[batch]
        states_ = self.new_state_memory[batch]
        rewards = self.reward_memory[batch]
        actions = self.action_memory[batch]
        terminal = self.terminal_memory[batch]
        
        return states, actions, rewards, states_, terminal
    
def build_dqn(lr, n_actions, input_dims, fc1_dims, fc2_dims):
    model = Sequential([
        Dense(fc1_dims, input_shape=(input_dims, )),
        Activation('relu'),
        Dense(fc2_dims),
        Activation('relu'),
        Dense(n_actions)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr), loss='mse')
        
    return model

In [12]:
class Agent(object):
    def __init__(self, alpha, gamma, n_actions, epsilon, batch_size, input_dims, epsilon_dec=0.9996, epsilon_end=0.01, mem_size=1000000, fname='dqn_model.h5'):
        self.action_space = [i for i in range(n_actions)]
        self.n_actions = n_actions
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_dec = epsilon_dec
        self.epsilon_min = epsilon_end
        self.batch_size = batch_size
        self.model_file = fname
        
        self.memory = ReplayBuffer(mem_size, input_dims, n_actions, discrete=True)
        self.q_eval = build_dqn(alpha, n_actions, input_dims, 256, 256)
        
    def remember(self, state, action, reward, new_state, done):
        self.memory.store_transition(state, action, reward, new_state, done)
        
    def choose_action(self, state):
        state = state[np.newaxis, :]
        rand = np.random.random()
        if rand < self.epsilon:
            action = np.random.choice(self.action_space)
        else:
            actions = self.q_eval.predict(state, verbose=0)
            action = np.argmax(actions)
        return action
    
    def learn(self):
        if self.memory.mem_cntr < self.batch_size:
            return 
        state, action, reward, new_state, done = self.memory.sample_buffer(self.batch_size)
        
        action_values = np.array(self.action_space, dtype=np.int8)
        action_indices = np.dot(action, action_values)
        
        q_eval = self.q_eval.predict(state, verbose=0)
        q_next = self.q_eval.predict(new_state, verbose=0)
        
        q_target = q_eval.copy()
        batch_index = np.arange(self.batch_size, dtype=np.int32)
        
        q_target[batch_index, action_indices] = reward + self.gamma*np.max(q_next, axis=1)*done
        _ = self.q_eval.fit(state, q_target, verbose=0)
        
        self.epsilon = self.epsilon * self.epsilon_dec if self.epsilon > self.epsilon_min else self.epsilon_min
        
    def save_model(self):
        self.q_eval.save(self.model_file)
        
    def load_model(self):
        self.q_eval = load_model(self.model_file)

In [13]:
if __name__ == 'main':
    env = gym.make('LunarLander-v2')
    n_games = 500
    agent = Agent(gamma=0.99, epsilon=1.0, alpha=0.0005, input_dims=8, n_actions=4, mem_size=1000000, batch_size=64, epsilon_end=0.01)
    scores = []
    eps_hist = []
    
    for i in range(n_games):
        done = False
        score = 0
        observation = env.reset()
        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            score += reward
            agent.remember(observation, action, reward, observation_, done)
            observation = observation_
            agent.learn()
        eps_hist.append(agent.epsilon)
        scores.append(score)
        
        avg_score = np.mean(scores[max(0, i-100):(i+1)])
        print('episode ', i, 'score %.2f' % score, 'average score %.2f' %avg_score)
        
        if i % 10 == 0 and i > 0:
            agent.save_model()
    

In [3]:
env = gym.make('LunarLander-v2')
#env = ChromeDinoEnv()
n_games = 500
agent = DDQNAgent(gamma=0.99, epsilon=1.0, alpha=0.0005, input_dims=env.observation_space.shape[0], n_actions=env.action_space.n, mem_size=1000000, batch_size=64, epsilon_end=0.01)
scores = []
eps_hist = []
    
for i in range(n_games):
    done = False
    score = 0
    observation = env.reset()
    while not done:
        action = agent.choose_action(observation)
        observation_, reward, done, info = env.step(action)
        score += reward
        agent.remember(observation, action, reward, observation_, done)
        observation = observation_
        agent.learn()
    eps_hist.append(agent.epsilon)
    scores.append(score)
        
    avg_score = np.mean(scores[max(0, i-100):(i+1)])
    print('episode ', i, 'score %.2f' % score, 'average score %.2f' %avg_score)
        
    if i % 10 == 0 and i > 0:
        agent.save_model()

NameError: name 'DDQNAgent' is not defined

In [ ]:
env = ChromeDinoEnv(render_mode='human')
done = False
observation = env.reset()
while not done:
    action = agent.choose_action(observation)
    observation_, reward, done, info = env.step(action)
    observation = observation_
    